# Spelling Corrector

In [80]:
import re
from collections import Counter


In [81]:
def edit_distance_1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletion    = [L + R[1:]               for L, R in splits if R]
    transposition = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    substitution   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    insertion    = [L + c + R               for L, R in splits for c in letters]
    return set(deletion + transposition + substitution + insertion)



In [82]:
len(edit_distance_1('ligt'))

234

In [83]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('minorproject.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

In [84]:
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

In [98]:
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edit_distance_1(word) for e2 in edit_distance_1(e1))

In [99]:
known(edit_distance_1('lamn'))

{'damn', 'lain', 'lamb', 'lame', 'lamp', 'lawn'}

In [100]:
known(edits2('longit'))

{'logic', 'login', 'long', 'longed', 'longer', 'longest', 'longing', 'longus'}

In [101]:
len(WORDS)

32198

In [102]:
sum(WORDS.values())

1115585

In [103]:
WORDS.most_common(20)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681),
 ('his', 10034),
 ('is', 9773),
 ('with', 9739),
 ('as', 8064),
 ('i', 7684),
 ('had', 7383),
 ('for', 6941),
 ('at', 6789),
 ('by', 6735),
 ('on', 6639)]

In [104]:
max(WORDS, key=P)

'the'

In [105]:
P('the')

0.07154004401278254

In [106]:
P('Prakhar')

0.0

In [107]:
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edit_distance_1(word)) or known(edits2(word)) or [word])

In [108]:
correction('griaf')

'grief'

In [113]:
def spelltest(tests, verbose=False):
    import time
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    print('{:.0%} of {} correct '.format(good / n, n))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

spelltest(Testset(open('test.txt')))

74% of 273 correct 
